---
layout: default
title: Individual Review IM
description: Recap of key events and learnings
type: tangibles
courses: { compsci: {week: 12} }
---

## Issue Recap
During the project, I have tried to keep issues to track work and progress.  

- [AWS Deployment ](https://github.com/iKAN2025/flask_portfolio/issues/1).   This highlights problems with completing AWS deployment/running in a docker container and deploying it using the AWS EC2 terminal and making it secure (https)
- [Fix Backend/Change JSON](https://github.com/iKAN2025/flask_portfolio/issues/4).   This is a quick discussion of how I decided to incorporate my backend code by giving the user a tracking attribute, an array with multiple tracking entries. 
- [Frontend Integration Code](https://github.com/iKAN2025/frontend/issues/8).  This was my ideas for our first integration checkpoint to fetch data from a database. I had problems iterating over the nested JSON (tracking attribute)
- [Update/Post](https://github.com/iKAN2025/flask_portfolio/issues/5).  This document demonstrates how I made the update/post methods work. When I used to the update/post method, I finally got rid of all local storage and simply fetched data from the database. Originally, I did not use the post method to create users, so everything was saved in local storage.
- [Graphs](https://github.com/iKAN2025/frontend/issues/10).  This is a demonstration of how I used practicedata to generate graohs,

## Key Commits


- [Fix Post Method ](https://github.com/nighthawkcoders/flask_portfolio/commit/38b940984af522b8b4c35e4ce4f0fc09cbe3b3dc). Fix Post Method/Update Method in backend
- [Fix Backend/Change JSON](https://github.com/nighthawkcoders/flask_portfolio/commit/fd636ea88e022d8eab82a1d08ed115855a6c56c4). Fix JSON(from ClOB TO JSON!!!) and update tracking attribute in model.py
- [Frontend Integration Code](https://github.com/iKAN2025/frontend/commit/9bb7d601d22238acfea539147be16c5440529ad0). Fix update method!
- [Nested JSON](https://github.com/iKAN2025/frontend/commit/ec47eac9c205756438f02a70b9ab30949bec4d29).  Iterating over nexted JSON

## Backend Code
``` python 
class UserAPI:        
    class _CRUD(Resource):  # User API operation for Create, Read.  THe Update, Delete methods need to be implemented
        def post(self): # Create method
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password and dob
            password = body.get('password')
            dob = body.get('dob')
            
            tracking = body.get('tracking') #validate tracking
            #

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, #user name
                      uid=uid, tracking=tracking)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # convert to date type
            # if dob is not None:
            #     try:
            #         uo.dob = datetime.strptime(dob, '%Y-%m-%d').date()
            #     except:
            #         return {'message': f'Date of birth format error {dob}, must be mm-dd-yyyy'}, 400
            if tracking is not None:
                uo.tracking = tracking
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                #return jsonify(user.read())
                return user.read()
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

        def get(self): # Read Method
            users = User.query.all()    # read/extract all users from database
            json_ready = [user.read() for user in users]  # prepare output in json
            #return jsonify(json_ready)  # jsonify creates Flask response object, more specific to APIs than json.dumps 
            return (json_ready) 
    class _SpecificUserID(Resource):        
        def put(self, user_id):
            body = request.get_json()
            user_id = body.get('id')
            if user_id is None:
                return {'message': 'Id not found.'}, 400
            user = User.query.filter_by(id=user_id).first()  # Use filter_by to query by UID
            if user:
                if 'tracking' in body:
                    user.tracking = body['tracking']
                    user.update()  # Update user in the database
                    return user.read()
                return {'message': 'You may only update tracking.'}, 400
            return {'message': 'User not found.'}, 404
        def get(self, user_id):
            user = User.query.filter_by(id=user_id).first()
            if user:
                return user.read()  # Assuming you have a 'read' method in your User model
            return {'message': 'User not found.'}, 404
            
            # body = request.get_json()
            # user_id = body.get('uid')
            # if user_id is None:
            #     return {'message': 'Id not found.'}, 400
            # user = User.query.get(id = user_id)
            # if body.get('tracking'):
            #     user.update(tracking = body.get('tracking')) 
            #     #return jsonify(user.read())
            #     return user.read()
            # return {'message': 'You may only update tracking.'}, 400
            
   
            

        
        
    class _Security(Resource):

        def post(self):
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Get Data '''
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            password = body.get('password')
            
            ''' Find user '''
            user = User.query.filter_by(_uid=uid).first()
            if user is None or not user.is_password(password):
                return {'message': f"Invalid user id or password"}, 400
            
            ''' authenticated user '''
            return jsonify(user.read())


    # building RESTapi endpoint
    api.add_resource(_CRUD, '/')
    api.add_resource(_SpecificUserID, '/<int:user_id>')
    api.add_resource(_Security, '/authenticate')
    

```


## Overview of Index PAGE - index.html
Main game page.  This has been an area of learning and rework. I created the carousel for the main page using bootstraps: 




```js

  <meta name="viewport" content="width=device-width, initial-scale=1">

  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css">

 


<div class="container"
>
  <div id="myCarousel" class="carousel slide" data-ride="carousel">
    <!-- Indicators -->
     <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.7.1/jquery.min.js"></script>
  
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/js/bootstrap.min.js"></script>
</head>
<body>

    <ol class="carousel-indicators">
      <li data-target="#myCarousel" data-slide-to="0" class="active"></li>
      <li data-target="#myCarousel" data-slide-to="1"></li>
      <li data-target="#myCarousel" data-slide-to="2"></li>
    </ol>
    <!-- Wrapper for slides -->
    <div class="carousel-inner">
      <div class="item active">
        <img src="./images/violin.jpeg" style="width:50rem;height:50rem">
      </div>
      <div class="item">
        <img src="./images/percussions.jpeg" style="width:50rem;height:50rem">
      </div>
     <div class="item">
        <img src="./images/oboe2.jpeg" style="width:50rem;height:50rem">
      </div>
      <div class="item">
        <img src="./images/synth.jpeg" style="width:50rem;height:50rem">
      </div>
    </div>
```



## CREATE USER PAGE-Post/Get Method 

-Uses Post Method to Create Users in the database
-Makes tracking array iterable to display user information in table
-Id is generated, so passed in random value
-uid is not generated-made random value
-post to api is an await function, waits for server to load DOM
-fetcges data from database that was sent and formats data in table


## Sends Data

```js
 <script>
         const url = "http://127.0.0.1:8240/api/users/"   
        // JavaScript to save practice time to local storage
            document.getElementById("save-button").addEventListener("click", async function () {
            //alert('Saving function');
            const practiceTime = document.getElementById("practice-time").value;
            const name = document.getElementById("name").value;
            const dob = document.getElementById("dob").value;
            const instrument = document.getElementById("instrument").value;
            const currentDate = new Date().toLocaleDateString();
            if (practiceTime !== "" && name !== "" && instrument !== "") {
                const practiceData = JSON.parse(localStorage.getItem(name)) || {};
                practiceData[currentDate] = {
                    dob: dob,
                    instrument: instrument,
                    time: parseInt(practiceTime),
                };
                localStorage.setItem(name, JSON.stringify(practiceData));
                //alert(`Practice time (${practiceTime} minutes) saved for ${currentDate}.`);
                if (parseInt(practiceTime) < 15) {
                    alert("Try to practice more tomorrow!");
                } else {
                    alert("Great job, keep it up!");
                }
                 // Create a JSON object for tracking data
                    var tracking_string = { 
                     "userName": name, 
                     "instrumentName": instrument,
                     "practiceDate": currentDate,
                     "practiceTime": practiceTime
                    };
                    let randm=Math.floor(Math.random() * 100) + 1;
                    let txtuid = name+ randm.toString();
                    //alert(txtuid);
                var data2 = {  //the other data that needs to be sent
                     "id": "3",
                     "name": name,
                     "uid": txtuid,  
                     "dob": dob,
                     "age": "16",
                     "tracking": tracking_string //tracking string
                };
                //alert(dob);
                 // Convert the JSON object to a string
               var jsonData = JSON.stringify(data2); //make data2 a string 
  // Call the API with the JSON payload
                try { //try and catch
                   // alert ('Calling the createUser Function');
                     await postToAPI(jsonData);
                   // await createUser(name, dob, instrument, currentDate, practiceTime);
                }
                catch (e) {
                    console.log(e) //log error
                }
                displayWeeklyLog(); //run function
            } else {
                alert("Please enter a valid practice time, name, and instrument.");
            }
        });
        // Function to display the weekly practice log
        //The fetch function is used to make an HTTP GET request to the URL 'https://bella-flask-portfolio.stu.nighthawkcodingsociety.com/api/users'. This is typically how you request data from a remote server using JavaScript.
        //The .then method is used to handle the response from the API request. It is a part of JavaScript's Promise-based architecture, which allows asynchronous operations to be handled in a more structured manner.
        //Inside the first .then block, the response from the API is converted to JSON format using response.json(). This is assuming that the response from the server is in JSON format.
        //The code then proceeds to update the content of the web page. It first retrieves a DOM element with the ID 'api-response' and a table body element inside a table with the ID 'weekly-log'. These elements are going to be used to display the data retrieved from the API.
function displayWeeklyLog() {
    fetch(url) // Replace 'url' with the actual API endpoint
        .then(response => response.json())
        .then(data => {
            const tableBody = document.querySelector("#weekly-log table tbody");
            tableBody.innerHTML = ""; // Clear the existing table data
            data.forEach(user => {
                if (Array.isArray(user.tracking)) {
                    user.tracking.forEach(trackingItem => {
                        const tracking = parseTracking(trackingItem);
                        const row = tableBody.insertRow();
                        const nameCell = row.insertCell(0);
                        const dobCell = row.insertCell(1);
                        const practicedateCell = row.insertCell(2);
                        const instrumentCell = row.insertCell(3);
                        const practicetimeCell = row.insertCell(4);
                        nameCell.textContent = user.name;
                        dobCell.textContent = user.dob;
                        practicedateCell.textContent = tracking.practiceDate;
                        instrumentCell.textContent = tracking.instrumentName;
                        practicetimeCell.textContent = tracking.practiceTime;
                    });
                }
            });
        })
        .catch(error => {
            console.error('Error:', error);
            alert(error);
        });
}
function parseTracking(tracking) {
    try {
        return JSON.parse(tracking);
    } catch (e) {
        return tracking;
    }
}
        displayWeeklyLog();
        // syntactic sugar approach!  is syntax within a programming language that is designed to make things easier to read or to express. It makes the language "sweeter" for human use: things can be expressed more clearly, more concisely, or in an alternative style that some may prefer.
        async function postToAPI(jsonData) {
            //alert(JSON.stringify(jsonData));
            //alert("Calling postToAPI");
            //alert(url);
            fetch(url, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: jsonData
            })
            .then(response => {
                if (response.ok) {
                    // Handle success
                    alert("Practice data has been added successfully");
                    console.log("Data sent successfully!");
                     document.getElementById("name").value = "";
                     document.getElementById("dob").value = "";
                     document.getElementById("practice-time").value = "";
                     document.getElementById("instrument").value = "";
                     displayWeeklyLog();
                } else {
                    // Handle errors
                    console.error("Failed to send data to the API.");
                    alert("Failed");
                     throw Error(`Error: ${response?.error?.data}`) ; //there's no guarantee respsonse data
                }
            })
           // .catch(error => {
                // Handle network errors
           //     console.error("Network error:", error);
           //     alert(error);
           // });
            //alert("end of postToAPI");
        }

```



## Tracker Page-Update/Get Method
uses full get method to  get all users from database
Use Update Method to update tracking data/and specific  get method to get old data from user and appened it to new tracking array-multiple entries

```js
 function populateNameDropdown() {
        // Make an API request to fetch user names
        // Replace this with your actual API endpoint
        fetch('http://127.0.0.1:8240/api/users/')
            .then(response => {
                if (!response.ok) {
                    throw new Error('Network response was not ok');
                }
                return response.json();
            })
            .then(data => {
                const nameDropdown = document.getElementById('name');
                data.forEach(user => {
                    const option = document.createElement('option');
                    option.value = user.id;
                    option.textContent = user.name;
                    nameDropdown.appendChild(option);
                });
            })
            .catch(error => {
                console.error('Error fetching user names:', error);
            });
    }
    populateNameDropdown()
   function updateTracking() {
    const selectedUserId = document.getElementById('name').value;
    const name = document.getElementById('name').options[document.getElementById('name').selectedIndex].text;
    const instrument = document.getElementById('instrument').value;
    const practiceTime = document.getElementById('practice-time').value;
    const currentDate = new Date().toLocaleDateString();
    if (practiceTime !== "" && name !== "" && instrument !== "") {
        fetch(`http://127.0.0.1:8240/api/users/${selectedUserId}`) //comment
            .then(response => {
                if (!response.ok) {
                    throw new Error('Network response was not ok');
                }
                return response.json();
            })
            .then(data => {
                // Combine old and new tracking data
                const originalTrackingData = Array.isArray(data.tracking) ? data.tracking : [];
                const newTrackingData = {
                    userName: name,
                    instrumentName: instrument,
                    practiceDate: currentDate,
                    practiceTime: practiceTime
                };
                const updatedTrackingData = [...originalTrackingData, newTrackingData];
                const data2 = {
                    "id": selectedUserId,
                    "name": name,
                    "uid": "life",
                    "dob": "10/12/13",
                    "age": "16",
                    "tracking": updatedTrackingData // Include updated tracking data
                };
                var jsonData = JSON.stringify(data2);
                fetch(`http://127.0.0.1:8240/api/users/${selectedUserId}`, {
                    method: 'PUT',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: jsonData
                }) //get method not working
                    .then(response => {
                        if (!response.ok) {
                            console.log('Network response was not ok');
                        }
                        return response.json();
                    })
                    .then(data => {
                        console.log('User tracking data updated:', data);
                    })
                    .catch(error => {
                        console.error('Error updating user tracking data:', error);
                    });
            })
            .catch(error => {
                console.error('Error fetching user data:', error);
            });
    } else {
        alert("Please enter a valid practice time, name, and instrument.");
    }
}
const url = "http://127.0.0.1:8240/api/users/"   
function displayWeeklyLog() {
    fetch(url) // Replace 'url' with the actual API endpoint
        .then(response => response.json())
        .then(data => {
            const tableBody = document.querySelector("#weekly-log table tbody");
            tableBody.innerHTML = ""; // Clear the existing table data
            data.forEach(user => {
                if (Array.isArray(user.tracking)) {
                    user.tracking.forEach(trackingItem => {
                        const tracking = parseTracking(trackingItem);
                        const row = tableBody.insertRow();
                        const nameCell = row.insertCell(0);
                        const dobCell = row.insertCell(1);
                        const practicedateCell = row.insertCell(2);
                        const instrumentCell = row.insertCell(3);
                        const practicetimeCell = row.insertCell(4);
                        nameCell.textContent = user.name;
                        dobCell.textContent = user.dob;
                        practicedateCell.textContent = tracking.practiceDate;
                        instrumentCell.textContent = tracking.instrumentName;
                        practicetimeCell.textContent = tracking.practiceTime;
                    });
                }
            });
        })
        .catch(error => {
            console.error('Error:', error);
            alert(error);
        });
}
function parseTracking(tracking) {
    try {
        return JSON.parse(tracking);
    } catch (e) {
        return tracking;
    }
}
        displayWeeklyLog();
document.addEventListener('DOMContentLoaded', function () {
    document.getElementById('save-button').addEventListener('click', function () {
        updateTracking();
    });
});



```




### Game Loop

This code is the heartbeat of the game.  All the Game Objects are refreshed.
- gameLoop is a function that is activated when page loads
- requestAnimationFrame recursively calls gameLoop according to refresh rate of computer screen

```js
// Game loop
function gameLoop() {
    for (var gameObj of GameObject.gameObjectArray){
        gameObj.update();
        gameObj.draw();
    }
    requestAnimationFrame(gameLoop);  // cycle game, aka recursion
}
```

### Event Listeners

The start file, in this case the index.html, often will often act on events produced by user.  

- Resize.  A common destop action is resize of window.   This will impact many Game Object as they are in proporition to size of window.

```js
// Window resize
window.addEventListener('resize', function () {
    GameEnv.setGameEnv();  // Update GameEnv dimensions

    // Call the sizing method on all game objects
    for (var gameObj of GameObject.gameObjectArray){
        gameObj.size();
    }
});

// Toggle "canvas filter property" between alien and normal
var isFilterEnabled = true;
const defaultFilter = getComputedStyle(document.documentElement).getPropertyValue('--default-canvas-filter');
toggleCanvasEffect.addEventListener("click", function () {
    for (var gameObj of GameObject.gameObjectArray){
        if (isFilterEnabled) {  // toggle off
            gameObj.canvas.style.filter = "none";  // remove filter
        } else { // toggle on
            gameObj.canvas.style.filter = defaultFilter;  // remove filter
        }
    }
    isFilterEnabled = !isFilterEnabled;  // switch boolean value
});
```

## Game environment - assets/js/alienWorld/GameEnv.js

A technique used in Object Oriented Programming is to handle system wide constant values in a "static" variables.  These variables are maintained in the "GameEnv" class and can be included and accessed with class name prefix.

In code in index.html you will see reference to GameEnv.speed and calls to set GamEnv.setGameEnv.  Properties are set in this code that enable objects to reference things like Top of screen, or ratio of Object speed versus GameSpeed.


Class definition of Global attributes

```js
export class GameEnv {
    // Prototype static variables
    static innerWidth;
    static prevInnerWidth;
    static innerHeight;
    static top;
    static bottom;
    static prevBottom;
    static gameSpeed;
    static gravity;

    // Make the constructor private to prevent instantiation
    constructor() {
        throw new Error('GameEnv is a static class and cannot be instantiated.');
    }

     // Setter for Top position
     static setTop() {
        // set top of game as header height
        const header = document.querySelector('header');
        if (header) {
            this.top = header.offsetHeight;
        }
    }

    // Setter for Bottom position
    static setBottom() {
        // set bottom of game as background height
        const background = document.querySelector('#background');
        if (background) {
            this.bottom = background.offsetHeight;
        }
    }
    
    // Setter for Game Environment 
    static setGameEnv() {
        // store previous for ratio calculatins on resize
        this.prevInnerWidth = this.innerWidth;
        this.prevBottom = this.bottom;
    
        // game uses available width and heith
        this.innerWidth = window.innerWidth;
        this.innerHeight = window.innerHeight;

        this.setTop();
        // this.setBottom() is ignored for now as resize of background object determinse bottom
    }
}

export default GameEnv;
```

## Graphs

Used Get Method to graph data. Mapped user data and got all of their practice times (user in a object, so user.tracking.PracticeTime (nested JSON))

```js
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tracker Data Visualization</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body>
    <div style="max-width: 800px; margin: 0 auto;">
        <canvas id="practiceChart"></canvas>
    </div>
</body>
<script>
    const apiUrl = 'http://127.0.0.1:8240/api/users/';
// Fetch user data from the API
fetch(apiUrl)
    .then(response => response.json())
    .then(data => {
        // Process your data and create datasets for the chart
        const users = data.map(user => user.name);
        const practiceTimes = data.map(user => user.tracking.practiceTime);
        // Create a chart using Chart.js
        const ctx = document.getElementById('practiceChart').getContext('2d');
        new Chart(ctx, {
            type: 'bar',
            data: {
                labels: users,
                datasets: [
                    {
                        label: 'Practice Time (minutes)',
                        data: practiceTimes,
                        backgroundColor: 'rgba(255, 105, 180, 0.2)', // Customize colors
                        borderColor: 'rgba(75, 192, 192, 1)', // Customize colors
                        borderWidth: 1,
                    }
                ]
            },
            options: {
                scales: {
                    y: {
                        beginAtZero: true,
                        title: {
                            display: true,
                            text: 'Minutes'
                        }
                    }
                }
            }
        });
    })
    .catch(error => {
        console.error('Error fetching data:', error);
    });
</script>
```
